# Lecture 11 Simple tool call with Langgraph must for stable tasks

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_API_KEY")
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from tavily import TavilyClient
import os

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(doc_content_chars_max = 1000000000))
tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

In [1]:
query="Indian Food"

In [5]:
from langchain_core.messages import AIMessage
from langchain_core.tools import tool

from langgraph.prebuilt import ToolNode

In [6]:
@tool
def get_wikipedia_result(query:str):
    "Call to get wikipedia result"
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(doc_content_chars_max = 10000000))
    return wikipedia.run(query)

@tool
def get_tavily_results(query:str):
    "Call to get tavily results"
    tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    response = tavily_client.search(query,max_results=10)
    content = ""
    for i in response['results']:
        content += i['content']
    return content

    

In [7]:
tool1=[get_wikipedia_result]
tool2=[get_tavily_results]
tool_node_1=ToolNode(tool1)
tool_node_2=ToolNode(tool2)


In [8]:
message_with_single_tool_call_wiki = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "get_wikipedia_result",
            "args": {"query": query},
            "id": "tool_call_id",
            "type": "tool_call",
        }
    ],
)

response_tool_1=tool_node_1.invoke({"messages": [message_with_single_tool_call_wiki]})

In [9]:
for m in response_tool_1['messages']:
     m.pretty_print()

================================= Tool Message =================================
Name: get_wikipedia_result

Page: Indian cuisine
Summary: Indian cuisine consists of a variety of regional and traditional cuisines native to the Indian subcontinent. Given the diversity in soil, climate, culture, ethnic groups, and occupations, these cuisines vary substantially and use locally available spices, herbs, vegetables, and fruits.
Indian food is also heavily influenced by religion, in particular Hinduism and Islam, cultural choices and traditions. Historical events such as invasions, trade relations, and colonialism have played a role in introducing certain foods to India. The Columbian discovery of the New World brought a number of new vegetables and fruits. A number of these such as potatoes, tomatoes, chillies, peanuts, and guava have become staples in many regions of India.
Indian cuisine has shaped the history of international relations; the spice trade between India and Europe was the pri

In [10]:
message_with_single_tool_call_tavily=AIMessage(
    content="",
        tool_calls=[
        {
            "name": "get_tavily_results",
            "args": {"query": query},
            "id": "tool_call_id",
            "type": "tool_call",
        }
    ],
)
response_tool_2=tool_node_2.invoke({"messages": [message_with_single_tool_call_tavily]})


In [11]:
for i in response_tool_2['messages']:
    i.pretty_print()

================================= Tool Message =================================
Name: get_tavily_results

Indian cuisine consists of a diverse range of curries, rice dishes, meats, vegetables and breads, all flavored with a traditional range of spices. Whilst delicacies vary by region and state, there are many similarities in terms of spices and flavors, regardless of the geographical region.Indian Food: 43 Traditional Dishes to Try in 2024 Dairy products such as yogurt, ghee, and paneer (Indian-style cheese) are commonly used in Indian cuisine and add richness and flavor to dishes. This bread can go with many other main dishes, so you will have many chances to try it whenever it comes to Indian foods. This South India-style rice dish is usually flavored with various Indian signature blended spices like oil-cooked jaggery, fried curry leaves, lentils, and peanuts. Considered one of the best starters from Asian cuisine, Indian tomato soup is a simple dish that will take you by surprise

In [12]:
sys_message_summarize=f'''You are expert at a giving a detailed summary of the content.
User Query:{query}

Wikipedia Results:{response_tool_1['messages']}

Tavily Results:{response_tool_2['messages']}

Always attribute responses to sources (Wikipedia and Tavily)

'''

In [13]:
final_summary=llm.invoke(sys_message_summarize)

In [14]:
print(final_summary.content)

Indian cuisine is incredibly diverse, varying significantly across its many regions due to differences in climate, soil, culture, and available ingredients (Wikipedia, Tavily).  This diversity is reflected in the wide array of spices, herbs, vegetables, and fruits used, leading to a vast range of flavors and textures (Wikipedia, Tavily).  Religious and cultural practices, particularly Hinduism and Islam, also significantly influence food choices and preparation (Wikipedia).

Historical events, including invasions, trade, and colonialism, have shaped Indian cuisine, introducing new ingredients like potatoes, tomatoes, chilies, peanuts, and guava (Wikipedia).  In fact, the spice trade between India and Europe was a major driver of the Age of Discovery (Wikipedia).  Indian cuisine's global impact is evident in its influence on cuisines worldwide, including those of Europe (especially Britain), the Middle East, Africa, Southeast Asia, North America, and the Caribbean (Wikipedia).  The WWF'

In [15]:
with open("results.txt", "w") as file:
    file.write(final_summary.content)